<img src="golden-gator.png" width="400">

# Godlen Gator notebook

## 1. setup 

### 1.1 Imports

In [1]:
import ast
import os

import networkx as nx
import pandas as pd
from typing import List
from Bio import SeqFeature, SeqIO

from dawdlib.embl_utils.embl_maker import create_dc_features, create_path_features
from dawdlib.degenerate_dna.deg_table import TableColNames, generate_deg_csv
from dawdlib.degenerate_dna.utils import parse_degenerate_codon_csv
from dawdlib.dijkstra import colorful
from dawdlib.dijkstra.len_limit import all_shortest_paths
from dawdlib.gg_dc_combine.gg_dc_combine import dc_df_codon_list, gate_cdn_oligos
from dawdlib.golden_gate.find_gg import deg_table_to_dict
from dawdlib.golden_gate.gate_data import GGData
from dawdlib.golden_gate.graph_maker import (
    GraphMaker,
    build_custom_graph,
    create_default_valid_node_function,
    create_default_weight_func,
    make_default_graph,
)
from dawdlib.golden_gate.reaction_sim import ReactionSim, ReactionGraphWt
from dawdlib.golden_gate.utils import RequirementsFactory, expand_dna_var_poss, parse_dna, check_for_restriction_sites

### 1.2 Constants and Paths

In [15]:
GG_TEMP = 25
GG_HOURS = 18
MIN_OLIGO_LENGTH: int = 20
MAX_OLIGO_LENGTH: int = 90
MIN_CONST_OLIGO_LENGTH: int = 20
MIN_NUM_GATES = 6
MAX_NUM_GATES = 20
MIN_EFFICIENCY = 0.25
MIN_FIDELITY = 0.9
CONST_COST = 40
RESTRICTION_ENZYME = ["BsaI"]
gatelength = 4
PREFIX = "GACATTGGTCTCA"
SUFFIX = "TGAGACCAACGACGCCGTACTCTTTGTCAAC"

    

reqs = RequirementsFactory(
    gg_temp = GG_TEMP,
    gg_hours = GG_HOURS,
    min_oligo_length = MIN_OLIGO_LENGTH,
    max_oligo_length = MAX_OLIGO_LENGTH,
    min_const_oligo_length = MIN_CONST_OLIGO_LENGTH,
    min_efficiency=MIN_EFFICIENCY,
    min_fidelity=MIN_FIDELITY,
    oligo_prefix=PREFIX,
    oligo_suffix=SUFFIX,
    const_cost = CONST_COST
)

W_PATH = "/Users/sh/Code/dawdlib/example"
resfile_path = os.path.join(W_PATH,"chosen_18Dec.resfile")
dna_path = os.path.join(W_PATH,"wt_dna.fasta")
embl_path = os.path.join(W_PATH,"wt_features_newgg.embl")
deg_table_path = os.path.join(W_PATH,"deg_table_newgg.csv")

ggdata = GGData(
    temperature=reqs.gg_temp,
    hours=reqs.gg_hours,
    min_efficiency=reqs.min_efficiency,
    min_fidelity=reqs.min_fidelity
)
dna = parse_dna(dna_path).upper()

### 1.2.1 Option to use 3 base pair gates instead of 4
### *Make sure to uncomment the next cell if you require using 3 base pairs instead of 4 or a different overhang data table from the ones provided built-in.*

In [3]:
# please make sure that you have changed the gatelength, RESTRICTION_ENZYME, PREFIX, and SUFFIX above
# path_to_ligation_data = "/home/labs/fleishman/arielte/dawdlib/dawdlib/golden_gate/resources/SapI.csv"
# set this path to be csv to ligation data
# ggdata.set_default_df(path_to_ligation_data)
# ggdata.init()

### Restriction enzyme verification

In [4]:
sites = check_for_restriction_sites(dna, RESTRICTION_ENZYME)
assert sites[0], f'Restriction enzyme {sites[1]} recognition site were found at positions {sites[2]} in the dna.'

## 2. Degenerate codons

### 2.1 Generate degenerate codon table

In [5]:
generate_deg_csv(resfile_path, csv_filename=deg_table_path)
deg_table = pd.read_csv(deg_table_path, na_filter=True, keep_default_na=False,)
encoded_diversity = deg_table.ENCODED_COUNT.apply(ast.literal_eval).apply(sum).prod()
print(f'The encoded diversity has {encoded_diversity} variants.')

The encoded diversity has 11059200 variants.


### 2.2 View degenerate codon table

In [6]:
deg_table

,AA_POS,DNA_POS,ENCODED_AAS,ENCODED_COUNT,AMBIGUOUS_CODONS1,AMBIGUOUS_CODONS2,AMBIGUOUS_CODONS3
0,16,46,"['I', 'V']","[1, 1]",RTT,,
1,42,124,"['L', 'V']","[1, 1]",STG,,
2,61,181,"['A', 'L', 'V']","[1, 1, 1]",STG,GCA,
3,65,193,"['S', 'T']","[1, 1]",ASC,,
4,68,202,"['A', 'M', 'V']","[1, 1, 1]",RTG,GCA,
5,69,205,"['A', 'L', 'P', 'Q']","[1, 1, 1, 1]",GCA,CHG,
6,72,214,"['A', 'C', 'S', 'T', 'V']","[1, 1, 1, 1, 1]",GTT,WGC,RCC
7,108,322,"['E', 'I', 'L', 'T', 'V']","[1, 1, 1, 1, 1]",GAA,AYT,STG
8,112,334,"['I', 'V']","[1, 1]",RTT,,
9,145,433,"['A', 'F', 'I', 'M', 'S', 'T', 'V', 'Y']","[1, 1, 1, 1, 1, 1, 1, 1]",TWT,RYG,AKT


## 3. Find golden gates

### 3.1 Create a graph
**Either use the default, or custom blocks.**

In [7]:
use_default = True

#### 3.1.1 Default graph

In [8]:
if use_default:
    gm = GraphMaker(ggdata)
    var_poss = expand_dna_var_poss(deg_table[TableColNames.DNA_POS.value].tolist())
    graph, src, target = make_default_graph(
        GraphMaker(ggdata), dna, var_poss, deg_table_to_dict(deg_table), reqs, gatelength
    )

#### 3.1.2 Custom graph

In [9]:
if not use_default:
    gm = GraphMaker(ggdata)
    var_poss = expand_dna_var_poss(deg_table[TableColNames.DNA_POS.value].tolist())

    is_valid_edge = gm.create_default_valid_edge_func(
        dna_var_poss=var_poss,
        min_oligo_length=MIN_OLIGO_LENGTH,
        max_oligo_length=MAX_OLIGO_LENGTH - len(PREFIX) - len(SUFFIX),
        min_const_oligo_length=MIN_CONST_OLIGO_LENGTH,
        min_fidelity=MIN_FIDELITY,
    )


    def cost_func(nd1, nd2):
        default = create_default_weight_func(
            dna_pos_n_codons=deg_table_to_dict(deg_table),
            oligo_addition=0,
            const_cost=0,
        )
        return default(nd1, nd2) + len(SUFFIX) + len(PREFIX)


    acceptable_fcws = ggdata.filter_self_binding_gates(filter_gc=True)
    is_valid_node = create_default_valid_node_function(acceptable_fcws, var_poss)

    graph, src, target = build_custom_graph(
        dna, is_valid_node, is_valid_edge, cost_func
    )

### 3.2 Find gates (shortest paths)

#### 3.2.1 Find shortest paths

In [30]:
shortest_paths = all_shortest_paths(
    graph, src, target, weight="weight", len_cutoff=MAX_NUM_GATES
)
best_paths = {}
bad_paths = {}
try:
    for i, (pth, cost) in enumerate(shortest_paths):
        rpth = [p for p in pth if not p.src_or_target]
        overhangs = [a.bps for a in rpth]
        try:
            reaction_fidelities = ggdata.reaction_fidelity(*overhangs)
        except ValueError:
            continue
        neb_fidelity = reaction_fidelities[0]
        if neb_fidelity > MIN_FIDELITY:
            try:
                if best_paths[len(rpth)][1] > cost:
                    best_paths[len(rpth)] = (pth, cost, neb_fidelity, i)
                    continue
                if best_paths[len(rpth)][2] < neb_fidelity:
                    best_paths[len(rpth)] = (pth, cost, neb_fidelity, i)
            except KeyError:
                best_paths[len(rpth)] = (pth, cost, neb_fidelity, i)
        else:
            try:
                bad_paths[len(rpth)].append(pth)
            except KeyError:
                bad_paths[len(rpth)] = [pth]
except nx.NetworkXNoPath:
    print(f"No path was found between {src} and {target}")

best_paths = dict((f'shortest_{i}', (p, c, g)) for p, c, g, i in best_paths.values())

##### 3.2.1.2 View found gates (paths)

In [1]:
for k, v in best_paths.items():
    print(f"Path ID: {k}. Number of gates: {len([a for a in v[0] if not a.src_or_target])}. Cost: {v[1]}.")

NameError: name 'best_paths' is not defined

#### 3.2.2 Find __*colorful*__ gates (shortest paths)

##### 3.2.2.1 Set colorful retries

In [18]:
colorful_retries = 100

In [19]:
spf = colorful.ShortestPathFinder(graph, ggdata, src, target)
for max_gates in range(MIN_NUM_GATES, MAX_NUM_GATES):
    for i in range(colorful_retries):
        pth = spf.find_shortest_path(len_cutoff=max_gates, no_colors=64)
        if pth:
            overhangs = [a.bps for a in pth[1:-1]]
            reaction_fidelity = ggdata.reaction_fidelity(*overhangs)
            if reaction_fidelity[0] < MIN_FIDELITY:
                continue
            pth_len = len(pth)
            cost = sum(
                (graph.edges[n1, n2]["weight"] for n1, n2 in zip(pth[:-1], pth[1:]))
            )
            entry = (pth, cost) + reaction_fidelity
            try:
                if best_paths[f"colorful_{max_gates}_gates"][1] > cost:
                    best_paths[f"colorful_{max_gates}_gates"] = entry
            except KeyError:
                best_paths[f"colorful_{max_gates}_gates"] = entry
            break

##### 3.2.2.2 View found *colorful* gates (paths)

In [20]:
for k, v in best_paths.items():
    print(f"Path ID: {k}. Number of gates: {len([a for a in v[0] if not a.src_or_target])}. Cost: {v[1]}.")

Path ID: colorful_17_gates. Number of gates: 16. Cost: 2121.
Path ID: colorful_18_gates. Number of gates: 16. Cost: 1773.
Path ID: colorful_19_gates. Number of gates: 16. Cost: 1787.


### 3.3 choose whichever path you want

In [21]:
chosen_entry = best_paths['colorful_18_gates']
chosen_path = chosen_entry[0]

In [22]:
chosen_path

[PseudoGate(idx=-1, bps='src', src_or_target=True, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=36, bps='CCCA', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=53, bps='GGAC', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=119, bps='CAAG', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=160, bps='CCGT', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=186, bps='ACCC', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=207, bps='TGCT', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=226, bps='ACCA', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=316, bps='ACAA', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=336, bps='AAGT', src_or_target=False, req_primer=False, syn_mut=(), gatelength=4),
 Gate(idx=426, bps='TACA', src_or_target=False, req_primer=False, syn

#### 3.3.1 save chosen path to csv

In [23]:
path_df = pd.DataFrame.from_records(chosen_path, columns=chosen_path[0].__annotations__.keys())
path_df.to_csv(os.path.join(W_PATH, 'chosen_path_newgg.csv'))

## 4. Create embl feature view

### 4.1 Prepare degenerate codon and gates features

In [24]:
deg_parsed_df = parse_degenerate_codon_csv(deg_table_path)
seq_features = create_dc_features(deg_parsed_df)
pth_features = create_path_features(chosen_path)

### 4.2 Save embl file with all features

In [25]:
seq_rec = SeqIO.read(dna_path, format="fasta")
seq_rec.annotations.update({"molecule_type": "DNA"})
seq_rec.features.extend(seq_features)
seq_rec.features.extend(pth_features)
SeqIO.write(seq_rec, embl_path, "embl")

1

## 5. Create oligos

### 5.1 Create oligo table

In [26]:
oligo_df = gate_cdn_oligos(chosen_path, dc_df_codon_list(deg_table), dna, reqs.oligo_prefix, reqs.oligo_suffix, "GFP-NEWGG")

### 5.2 Save oligo table

In [27]:
oligo_df.to_csv(os.path.join(W_PATH, "oligo_df_newgg.csv"))

## 6. Verify golden gate reaction

### 6.1 Create golden gate simulator and load oligo table

In [28]:
rs = ReactionSim(ggdata, reqs, RESTRICTION_ENZYME)
res = rs.create_reaction_graph(os.path.join(W_PATH, "oligo_df_newgg.csv"))
if res is not None:
    msg = res[0]
    oligo_entry = res[1]
    print(msg)
    print(f'The choice of degenerate codons {oligo_entry.oligo_codons} in oligo named "{oligo_entry.name}" created a new enzyme restriction site!')
    print('''This error must be resolved manually!
open the file “deg_table.csv” that was created by box 2.2 and find the relevant segment by the name given above.
Try to edit the selected codons to eliminate the creation of the BsaI site.
Then, comment out the first line in box 2.2 (which created deg_table.csv) and now rerun the notebook again from box 2.2.
The notebook will use your edited file without the enzyme restriction site.''')


ValueError: max() arg is an empty sequence

### 6.2 Check for WT sequence

In [29]:
reaction_wts:List[ReactionGraphWt] = list(rs.get_wt_dna())
assert 1 == len(reaction_wts), "Error: {len(reaction_wt)} WT DNA sequences found! expected 1!!"
reaction_wt = reaction_wts[0]
assert reaction_wt.dna == dna, "Error: reaction DNA doesn't match input DNA!!!"

AttributeError: 'ReactionSim' object has no attribute 'reaction_graph'

### 6.3 Verify all golden gate products
**Checks that all products are constructed correctly and have the same length and gates as WT**

* Note: This might take a while!

In [30]:
result = rs.verify_reaction(reaction_wt.end - reaction_wt.start, reaction_wt.no_segments, reaction_wt.fidelity_sum)
if result[0]:
    msg = '\n'.join([f'The diversity of the degenerate table ({encoded_diversity}) differs from the one found by the simulation {result[1]}',
           'Do not continue or use the product of this run! (unless you know exactly what you\'re doing',
           'Either the golden gate reaction failed, a restriction site appeared or some of the diversity disappeared somewhere!'])
    assert result[1] == encoded_diversity, msg
    print(f"Golden gate simulation passed! the number of different products is {result[1]}")
else:
    print("Verifying golden gate reaction failed!!!")
    print("The following product failed verification:\n")
    print(result[1])

Golden gate simulation passed! the number of different products is 11059200


## 7. Write order table

### 7.1 Settings

#### Write constant segments?

In [23]:
output_const = False

#### write WT segments?

In [24]:
output_wt = False

### Write order table to csv file

In [25]:
oligo_df[(oligo_df.wt <= output_wt) & (oligo_df.const <= output_const)][['name', 'full_oligo_dna']].to_csv(os.path.join(W_PATH, "order_table_newgg.csv"))